In [310]:
using StaticArrays
using LinearAlgebra
using Symbolics
using Latexify


In [2]:
using Pkg

In [3]:
Pkg.status()

Project project_1 v0.1.0
Status `~/Documents/linierAlgebra/julia/project_1/Project.toml`
  [a93c6f00] DataFrames v1.7.0
  [23fbe1c1] Latexify v0.16.5
  [91a5bcdd] Plots v1.40.9
  [90137ffa] StaticArrays v1.9.8
  [0c5d862f] Symbolics v6.22.0
  [37e2e46d] LinearAlgebra v1.11.0


In [4]:
v_1 = [14, 30, -5]
v_2 = [23, 56, 2]
@variables a b c d

4-element Vector{Num}:
 a
 b
 c
 d

In [5]:
M = [v_1 v_2]'

2×3 adjoint(::Matrix{Int64}) with eltype Int64:
 14  30  -5
 23  56   2

In [6]:
I = M[1:2]

2-element Vector{Int64}:
 14
 23

In [7]:
J = M[3:4]

2-element Vector{Int64}:
 30
 56

In [8]:
K = M[5:6]

2-element Vector{Int64}:
 -5
  2

In [9]:
[K J]

2×2 Matrix{Int64}:
 -5  30
  2  56

In [10]:
I .* [J K] .+ (-J) .* [I K] .+ K .* [I J]

2×2 Matrix{Int64}:
 -70  -70
  46   46

In [11]:
prod(J)

1680

In [12]:
Sdeteminat(M_mn) = substitute(a * b - c * d, Dict(a => M_mn[1], b => M_mn[4], c => M_mn[3], d => M_mn[2]))
@register_symbolic Sdeteminat(M_mn)

In [13]:
crossProduct = [Sdeteminat([J K]), Sdeteminat([I K]), Sdeteminat([I J])]

3-element Vector{Num}:
 340
 143
  94

In [14]:
M = [a; b]

2-element Vector{Num}:
 a
 b

In [15]:
c = M[1:1]
c

1-element Vector{Num}:
 a

In [16]:
Symbolics.value(c)

1-element Vector{Num}:
 a

In [17]:
@variables M[[3, 15, 7]', [-41, 27, 10]']

1-element Vector{Symbolics.Arr{Num, 2}}:
 M[[3 15 7],[-41 27 10]]

In [18]:
using Symbolics
@variables x
quintic = x^5 + x ~ 1

x + x^5 ~ 1

In [19]:
function solve_newton(eq, x, x₀; abstol=1e-8, maxiters=50)
    # symbolic expressions for f(x) and f′(x)
    f = eq.lhs - eq.rhs # want to find root of f(x)
    f′ = Symbolics.derivative(f, x)

    xₙ = x₀ # numerical value of the initial guess
    for i = 1:maxiters
        # calculate new guess by numerically evaluating symbolic expression at previous guess
        xₙ₊₁ = substitute(x - f / f′, x => xₙ)
        if abs(xₙ₊₁ - xₙ) < abstol
            return xₙ₊₁ # converged
        else
            xₙ = xₙ₊₁
        end
    end
    error("Newton's method failed to converge")
end

x_newton = solve_newton(quintic, x, 1.0)
println("Newton's method solution: x = ", x_newton)


Newton's method solution: x = 0.7548776662466927


In [20]:
@variables ϵ # expansion variable
quintic = x^5 + ϵ*x ~ 1


x*ϵ + x^5 ~ 1

In [21]:
x_coeffs, = @variables a[0:7] # create Taylor series coefficients
x_taylor = series(x_coeffs, ϵ) # expand x in a power series in ϵ


a[0] + a[1]*ϵ + a[2]*(ϵ^2) + a[3]*(ϵ^3) + a[4]*(ϵ^4) + a[5]*(ϵ^5) + a[6]*(ϵ^6) + a[7]*(ϵ^7)

In [22]:
quintic_taylor = substitute(quintic, x => x_taylor)
quintic_taylor = taylor(quintic_taylor, ϵ, 0:7)


a[0]^5 + (a[0] + 5(a[0]^4)*a[1])*ϵ + (1//2)*(2a[1] + 10(a[0]^4)*a[2] + 20(a[0]^3)*(a[1]^2))*(ϵ^2) + (1//6)*(6a[2] + 30(a[0]^4)*a[3] + 120(a[0]^3)*a[1]*a[2] + 60(a[0]^2)*(a[1]^3))*(ϵ^3) + (1//24)*(24a[3] + 120(a[0]^4)*a[4] + 480(a[0]^3)*a[1]*a[3] + (240//1)*(a[0]^3)*(a[2]^2) + 720(a[0]^2)*(a[1]^2)*a[2] + 120a[0]*(a[1]^4))*(ϵ^4) + (1//120)*(120a[4] + 600(a[0]^4)*a[5] + 2400(a[0]^3)*a[1]*a[4] + 2400(a[0]^3)*a[2]*a[3] + 3600(a[0]^2)*(a[1]^2)*a[3] + (3600//1)*(a[0]^2)*a[1]*(a[2]^2) + 2400a[0]*(a[1]^3)*a[2] + 120(a[1]^5))*(ϵ^5) + (1//720)*(720a[5] + 3600(a[0]^4)*a[6] + 14400(a[0]^3)*a[1]*a[5] + 14400(a[0]^3)*a[2]*a[4] + (7200//1)*(a[0]^3)*(a[3]^2) + 21600(a[0]^2)*(a[1]^2)*a[4] + 43200(a[0]^2)*a[1]*a[2]*a[3] + (7200//1)*(a[0]^2)*(a[2]^3) + 14400a[0]*(a[1]^3)*a[3] + (21600//1)*a[0]*(a[1]^2)*(a[2]^2) + 3600(a[1]^4)*a[2])*(ϵ^6) + (1//5040)*(5040a[6] + 25200(a[0]^4)*a[7] + 100800(a[0]^3)*a[1]*a[6] + 100800(a[0]^3)*a[2]*a[5] + 100800(a[0]^3)*a[3]*a[4] + 151200(a[0]^2)*(a[1]^2)*a[5] + 302400(a[0]^2

In [23]:
quintic_eqs = taylor_coeff(quintic_taylor, ϵ, 0:7)
quintic_eqs[1:5] # for readability, show only 5 shortest equations


5-element Vector{Equation}:
 a[0]^5 ~ 1//1
 a[0] + 5(a[0]^4)*a[1] ~ 0//1
 (1//2)*(2a[1] + 10(a[0]^4)*a[2] + 20(a[0]^3)*(a[1]^2)) ~ 0//1
 (1//6)*(6a[2] + 30(a[0]^4)*a[3] + 120(a[0]^3)*a[1]*a[2] + 60(a[0]^2)*(a[1]^3)) ~ 0//1
 (1//24)*(24a[3] + 120(a[0]^4)*a[4] + 480(a[0]^3)*a[1]*a[3] + (240//1)*(a[0]^3)*(a[2]^2) + 720(a[0]^2)*(a[1]^2)*a[2] + 120a[0]*(a[1]^4)) ~ 0//1

In [24]:
function solve_cascade(eqs, xs, x₀, ϵ)
    sol = Dict(xs[begin] => x₀) # store solutions in a map

    # verify that x₀ is a solution of the first equation
    eq0 = substitute(eqs[1], sol)
    isequal(eq0.lhs, eq0.rhs) || error("$sol does not solve $(eqs[1])")

    # solve remaining equations sequentially
    for i in 2:length(eqs)
        eq = substitute(eqs[i], sol) # insert previous solutions
        x = xs[begin+i-1] # need not be 1-indexed
        xsol = Symbolics.symbolic_linear_solve(eq, x) # solve current equation
        sol = merge(sol, Dict(x => xsol)) # store solution
    end

    return sol
end


solve_cascade (generic function with 1 method)

In [25]:
x_coeffs_sol = solve_cascade(quintic_eqs, x_coeffs, 1, ϵ)
x_pert = substitute(x_taylor, x_coeffs_sol)


1 - (1//5)*ϵ - (1//25)*(ϵ^2) - (1//125)*(ϵ^3) + (21//15625)*(ϵ^5) + (78//78125)*(ϵ^6) + (187//390625)*(ϵ^7)

In [26]:
for n in 0:7
    x_pert_sol = substitute(taylor(x_pert, ϵ, 0:n), ϵ => 1)
    println("$n-th order solution: x = $x_pert_sol = $(x_pert_sol * 1.0)")
end


0-th order solution: x = 1//1 = 1.0
1-th order solution: x = 4//5 = 0.8
2-th order solution: x = 19//25 = 0.76
3-th order solution: x = 94//125 = 0.752
4-th order solution: x = 94//125 = 0.752
5-th order solution: x = 11771//15625 = 0.753344
6-th order solution: x = 58933//78125 = 0.7543424
7-th order solution: x = 294852//390625 = 0.75482112


# Dot product!

In [30]:
@variables u v
u_1 = [3, 1 ,4]
v_2 = [1, -5 ,0]

3-element Vector{Int64}:
  1
 -5
  0

In [28]:
Symbolics.sum(u * v)

u*v

In [31]:
dotProduct(u, v) = substitute(sum(v .* u), Dict(u => u, v => v ))
@register_symbolic dotProduct(u, v)

In [33]:
dotProduct(u_1, v_2)

-2

*Magnitude or lenght.*

In [47]:
magnitude(u) = substitute(sqrt(dotProduct(u, u) * dotProduct(u, u)), (u => u))
@register_symbolic magnitude(u)

In [48]:
magnitude(u_1)

26.0

In [61]:
@variables A[1:5, 1:3] b[1:3]


2-element Vector{Symbolics.Arr{Num}}:
 A[1:5,1:3]
 b[1:3]

In [67]:
Symbolics.scalarize(b[1:3])

3-element Vector{Num}:
 b[1]
 b[2]
 b[3]

In [70]:
substitute(Symbolics.scalarize(b[1:3]) * 2, Dict(b1 => 1, b2 => 2, b3 => 3))

UndefVarError: UndefVarError: `b1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

*Some Matrix function and properties.*

In [71]:
A = [1 2 3
    4 5 6 ]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [72]:
I = [1 0 0
     0 1 0 
     0 0 1]

3×3 Matrix{Int64}:
 1  0  0
 0  1  0
 0  0  1

In [73]:
A * I 

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [77]:
Z = (0) 

0

In [82]:
A * Z

2×3 Matrix{Int64}:
 0  0  0
 0  0  0

In [83]:
A + (-A)

2×3 Matrix{Int64}:
 0  0  0
 0  0  0

In [85]:
A'

3×2 adjoint(::Matrix{Int64}) with eltype Int64:
 1  4
 2  5
 3  6

In [91]:
(A')' == A

true

In [88]:
B = [7 8 9
     10 11 12]

2×3 Matrix{Int64}:
  7   8   9
 10  11  12

In [89]:
(A + B)' == A' + B'

true

In [94]:
(5A)' == 5A'

true

In [106]:
(A .* B)' == B' .* A'

true

## Symbolic Array variables!

In [108]:
Symbolics.variables(:x, 4, 2, 0)

0-dimensional Array{Num, 0}:
x₄ˏ₂ˏ₀

In [114]:
C = x * 2

2x

In [130]:
X = Symbolics.variables(:x, 1:3, 3:6)

3×4 Matrix{Num}:
 x₁ˏ₃  x₁ˏ₄  x₁ˏ₅  x₁ˏ₆
 x₂ˏ₃  x₂ˏ₄  x₂ˏ₅  x₂ˏ₆
 x₃ˏ₃  x₃ˏ₄  x₃ˏ₅  x₃ˏ₆

In [133]:
c = X * 2

3×4 Matrix{Num}:
 2x₁ˏ₃  2x₁ˏ₄  2x₁ˏ₅  2x₁ˏ₆
 2x₂ˏ₃  2x₂ˏ₄  2x₂ˏ₅  2x₂ˏ₆
 2x₃ˏ₃  2x₃ˏ₄  2x₃ˏ₅  2x₃ˏ₆

In [152]:
c = Symbolics.scalarize( 3 * sum( X[3:6]))

3(x₁ˏ₄ + x₂ˏ₄ + x₃ˏ₃ + x₃ˏ₄)

**With the subscrite index clear I use substitution and solve any system of lenier equations**

In [179]:
Symbolics.substitute(c, Dict(X[:4] => 1, X[:3] => 2, X[:5] => 3, X[:6] => 4))

30

In [192]:
@variables A[1:3 ,4:6 ]

1-element Vector{Symbolics.Arr{Num, 2}}:
 A[1:3,4:6]

In [236]:
c = A[1:3, 4:6] * 2 

(broadcast(*, A[1:3, 4:6], 2))[Base.OneTo(3),Base.OneTo(3)]

In [242]:
@variables t x y z(t);

Symbolics.get_variables(x + y + sin(z); sort = true)




3-element Vector{SymbolicUtils.BasicSymbolic}:
 x
 y
 z(t)

In [254]:
Symbolics.get_variables(A; sort = true) 


UndefVarError: UndefVarError: `get_variables` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [333]:
@variables x a b
expr = Symbolics.expand((x + b)*(x^2 + 2x + 1)*(x^2 - a))

-a*b - a*x - 2a*b*x - 2a*(x^2) + b*(x^2) + x^3 - a*b*(x^2) - a*(x^3) + 2b*(x^3) + 2(x^4) + b*(x^4) + x^5

In [338]:
A = [1 2; 3 4]


tr(A)

5

In [ ]:
.√(A)

2×2 Matrix{Float64}:
 1.0      1.41421
 1.73205  2.0

**Buiding function to be used as symbolic or numerical in julia**

In [ ]:
to_compute = [x^2 + y, y^2 + x]
f_expr = build_function(expr, [x, A])

In [ ]:
myf = eval(f_expr)

In [ ]:
values(myf([2, 2]))